In [2]:
# import necessary library
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

In [1]:
def load_dataframe(path=None, nrows=100000):
    if not path:
        path = '~/data_analysis/data/tappware/dnothi/employee_records.csv'
    return pd.read_csv(path, nrows=nrows)

In [3]:
df = load_dataframe()
df.shape

/tmp/ipykernel_103837/4292819960.py:4: DtypeWarning: Columns (8,17,18,23,25,34,36,39,45,46,47) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(path, nrows=nrows)


(100000, 52)

In [4]:
df.head()

,id,name_eng,name_bng,father_name_eng,father_name_bng,mother_name_eng,mother_name_bng,date_of_birth,nid,nid_valid,bcn,ppn,gender,religion,blood_group,marital_status,personal_email,personal_mobile,alternative_mobile,is_cadre,employee_grade,employee_cadre_id,employee_batch_id,identity_no,appointment_memo_no,joining_date,service_rank_id,service_grade_id,service_ministry_id,service_office_id,current_office_ministry_id,current_office_layer_id,current_office_id,current_office_unit_id,current_office_joining_date,current_office_designation_id,current_office_address,e_sign,d_sign,image_file_name,status,default_sign,hard_signature,soft_signature,cert_id,cert_type,cert_provider,cert_serial,created_by,modified_by,created,modified
0,30,Maherunnesa,মেহেরুন নেছা,NaN,NaN,NaN,NaN,1958-07-09 00:00:00,3923606142583,0,NaN,NaN,NaN,NaN,NaN,NaN,meherunnisa.jamalpur@gmail.com,01717-090607,NaN,2,NaN,0.0,0.0,0,NaN,0000-00-00 00:00:00,0.0,13.0,5.0,NaN,0.0,0.0,0.0,0.0,0000-00-00 00:00:00,0.0,"জেলা প্রশাসকের কার্যালয়, জামালপুর",NaN,NaN,NaN,0,0,0,0,NaN,NaN,NaN,NaN,0,0,0000-00-00 00:00:00,0000-00-00 00:00:00
1,31,Md. Raziul Hassan,মো: রাজিউল হাসান,Let.Manir udden sarkar,মৃত: মনির উদ্দিন সরকার,Let.Bashironnasa,মৃত: বছিরননেছা,1963-11-02 00:00:00,19633923602111523,0,NaN,NaN,1.0,Islam,B+,Married,raziul.hasan57@gmail.com,01715-566157,01713545338,2,NaN,0.0,0.0,0,NaN,1984-09-08 00:00:00,0.0,13.0,51.0,NaN,0.0,0.0,0.0,0.0,0000-00-00 00:00:00,0.0,জেলা ত্রাণ ও পুনর্বাসন কর্মকর্তার কার্যালয়,NaN,NaN,NaN,0,0,0,0,NaN,NaN,NaN,NaN,0,0,0000-00-00 00:00:00,2016-10-25 12:59:33
2,34,Md. Abul Hossain,মো: আবুল হোসাইন,Late Abul kashem,মৃত আবুল কাশেম,Late Amena Khatun.,মৃত আমেনা খাতুন,1958-04-21 00:00:00,19583923604125262,0,NaN,NaN,1.0,Islam,O-,Married,Abul Hossain@gmail.com,01718191256,NaN,2,NaN,0.0,0.0,0,NaN,1987-10-11 00:00:00,0.0,14.0,5.0,NaN,0.0,0.0,0.0,0.0,0000-00-00 00:00:00,0.0,"জেলা প্রশাসকের কার্যালয়, জামালপুর",NaN,NaN,NaN,0,0,0,0,NaN,NaN,NaN,NaN,0,0,0000-00-00 00:00:00,2016-11-27 16:35:01
3,37,Md. Sarwardi,মো: ছরওয়ারদি,NaN,NaN,NaN,NaN,2009-02-03 00:00:00,20093923602110054,0,NaN,NaN,NaN,NaN,NaN,NaN,m_sarwardi@yahoo.com,01679-061448,NaN,2,NaN,0.0,0.0,0,NaN,0000-00-00 00:00:00,0.0,13.0,31.0,NaN,0.0,0.0,0.0,0.0,0000-00-00 00:00:00,0.0,"জেলা প্রশাসকের কার্যালয়, জামালপুর",NaN,NaN,NaN,0,0,0,0,NaN,NaN,NaN,NaN,0,0,0000-00-00 00:00:00,2016-11-29 17:34:47
4,39,Md. Sultan Uddin,মো: সুলতান উদ্দিন,NaN,NaN,NaN,NaN,1962-10-31 00:00:00,19623913695446315,0,NaN,NaN,NaN,NaN,NaN,NaN,sultan_nazir43@yahoo.com,01714-711222,NaN,2,NaN,0.0,0.0,0,NaN,1987-10-11 00:00:00,0.0,13.0,5.0,NaN,0.0,0.0,0.0,0.0,0000-00-00 00:00:00,0.0,"জেলা প্রশাসকের কার্যালয়, জামালপুর",NaN,NaN,NaN,0,0,0,0,NaN,NaN,NaN,NaN,0,0,0000-00-00 00:00:00,2016-11-11 09:39:37


In [8]:
df.columns

Index(['id', 'name_eng', 'name_bng', 'father_name_eng', 'father_name_bng',
       'mother_name_eng', 'mother_name_bng', 'date_of_birth', 'nid',
       'nid_valid', 'bcn', 'ppn', 'gender', 'religion', 'blood_group',
       'marital_status', 'personal_email', 'personal_mobile',
       'alternative_mobile', 'is_cadre', 'employee_grade', 'employee_cadre_id',
       'employee_batch_id', 'identity_no', 'appointment_memo_no',
       'joining_date', 'service_rank_id', 'service_grade_id',
       'service_ministry_id', 'service_office_id',
       'current_office_ministry_id', 'current_office_layer_id',
       'current_office_id', 'current_office_unit_id',
       'current_office_joining_date', 'current_office_designation_id',
       'current_office_address', 'e_sign', 'd_sign', 'image_file_name',
       'status', 'default_sign', 'hard_signature', 'soft_signature', 'cert_id',
       'cert_type', 'cert_provider', 'cert_serial', 'created_by',
       'modified_by', 'created', 'modified'],
      dtype

## Ideas
- Email ব্যবহারকারীর অনুপাত (Gmail, yahoo, ...)
- কতজন ব্যবহারকারীর ইমেইল দেওয়া হয় নাই
- কোন মাস এ Birthday বেশি
- কোন দিন এ Birthday বেশি
- Blood group অনুপাত।
- Blood group , Male Female এর মধ্যে Relationships.
- বিবাহিত ও অবিবাহিত ব্যবহারকারীর অনুপাত
- কোন Month এ জইনিং বেশি।
- ব্যবহারকারীরা কোন মোবাইল অপারেটর বেশি ব্যবহার করেন
- কতজন ব্যবহারকারী marital status গোপণ রেখেছেন। :)



In [38]:
df_email = df[~df.personal_email.isnull()].copy(deep=True)
# df_email = df.personal_email
# dir(df_email.str)
df_email['email_domain'] = df_email['personal_email'].str.extract(r'@(\w+)\.')
# df_email.plot(kind='bar')


In [44]:
df_email.personal_email.str.split('@')

0        [meherunnisa.jamalpur, gmail.com ]
1              [raziul.hasan57, gmail.com ]
2                 [Abul Hossain, gmail.com]
3                  [m_sarwardi, yahoo.com ]
4               [sultan_nazir43, yahoo.com]
                        ...                
99995        [nuruzzaman68.sgfl, gmail.com]
99996               [ch.samir71, gmail.com]
99997               [anarul0090, gmail.com]
99998                [amptg.pub, gmail.com]
99999              [salam010169, gmail.com]
Name: personal_email, Length: 88434, dtype: object

In [39]:
df_email.groupby('email_domain').size().reset_index(name='counts')

,email_domain,counts
0,2yahoo,1
1,30yahoo,1
2,589,1
3,60gmail,1
4,65gmail,1
...,...,...
471,zpmehepur,1
472,zpmeherpur,1
473,zpnaogaon,1
474,zppatuakhali,1
